# Zomato Spatial Analysis

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('Dataset/zomato.csv')

In [5]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [6]:
df.isna().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [7]:
# dropping empty location rows because i a m going to do spatial analysis and for that location must not be empty
df.dropna(axis=0, subset=['location'], inplace=True)

In [8]:
df.isna().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7754
votes                              0
phone                           1187
location                           0
rest_type                        206
dish_liked                     28057
cuisines                          24
approx_cost(for two people)      325
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [10]:
len(df['location'].unique())

93

In [11]:
print(df['location'].unique())

['Banashankari' 'Basavanagudi' 'Mysore Road' 'Jayanagar'
 'Kumaraswamy Layout' 'Rajarajeshwari Nagar' 'Vijay Nagar' 'Uttarahalli'
 'JP Nagar' 'South Bangalore' 'City Market' 'Nagarbhavi'
 'Bannerghatta Road' 'BTM' 'Kanakapura Road' 'Bommanahalli'
 'CV Raman Nagar' 'Electronic City' 'HSR' 'Marathahalli' 'Sarjapur Road'
 'Wilson Garden' 'Shanti Nagar' 'Koramangala 5th Block'
 'Koramangala 8th Block' 'Richmond Road' 'Koramangala 7th Block'
 'Jalahalli' 'Koramangala 4th Block' 'Bellandur' 'Whitefield'
 'East Bangalore' 'Old Airport Road' 'Indiranagar' 'Koramangala 1st Block'
 'Frazer Town' 'RT Nagar' 'MG Road' 'Brigade Road' 'Lavelle Road'
 'Church Street' 'Ulsoor' 'Residency Road' 'Shivajinagar' 'Infantry Road'
 'St. Marks Road' 'Cunningham Road' 'Race Course Road' 'Commercial Street'
 'Vasanth Nagar' 'HBR Layout' 'Domlur' 'Ejipura' 'Jeevan Bhima Nagar'
 'Old Madras Road' 'Malleshwaram' 'Seshadripuram' 'Kammanahalli'
 'Koramangala 6th Block' 'Majestic' 'Langford Town' 'Central Bangalore'


In [12]:
# creating blank dataframe
locations = pd.DataFrame()

In [13]:
# adding unique locations(cities) in my new dataframe
locations['Name'] = df['location'].unique()
locations.head()

,Name
0,Banashankari
1,Basavanagudi
2,Mysore Road
3,Jayanagar
4,Kumaraswamy Layout


#### Extracting Latitudes and Longitudes of each unique locations

In [14]:
# uncomment this line to install library
#!pip install geopy

In [15]:
from geopy.geocoders import Nominatim

In [16]:
# establishing connection
geolocator = Nominatim(user_agent='app')

In [17]:
lat = []
lon = []
for location in locations['Name']:
    
    # converting string location into geo format
    location = geolocator.geocode(location)
    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)

In [18]:
print(lat)

[15.8876779, 12.9417261, 12.3872141, 27.64392675, 12.9081487, 12.9274413, 22.8359967, 12.9055682, 12.2655944, 13.0646907, 39.76880625, 12.9546741, 12.887979, 45.95485055, 12.5607431, 12.9089453, 17.2510682, -6.2659285, 18.1475, 12.9552572, 12.9242381, 12.9489339, 12.9575547, 13.00540865, 12.9417812, 50.7721586, 13.0078418, 11.9917786, 12.93433385, 12.93577245, 44.3730577, 13.0215466, 38.4876975, 12.9732913, 14.5395813, 12.996845, 13.0227204, 17.9895618, 40.28745, 40.7652844, 51.373656, 12.9778793, 38.7385916, 18.5322493, 34.977289, 51.5227651, 31.89376, 22.3146438, 51.5173375, 12.988721250000001, 13.0358698, 12.9624669, 12.945245, 12.9678074, 13.194416, 13.0027353, 12.9931876, 13.0093455, 12.9400321, 1.2847055, 12.957998, 13.0101286, 23.1485712, 33.5935063, 12.967576, 12.9414662, 13.007516, 12.9243509, 12.9271867, 13.2923988, 12.773175, 12.9882338, 13.0141618, 13.0217151, 13.2227, 13.0258087, 13.0221416, 13.0346384, 13.0784743, nan, 12.973936, 12.9846713, 13.0382184, 12.9176571, 38.780

In [19]:
print(lon)

[75.7046777, 77.5755021, 76.6669626, 83.05280519687284, 77.5553179, 77.5155224, 69.3405962, 77.5455438, 76.6465404, 77.49626895712257, -86.15345077251979, 77.5121724, 77.5970812, -112.49659530324134, 77.4258375, 77.6239038, 80.1651978, 106.7842561, 41.538889, 77.6984163, 77.6289059, 77.5968273, 77.5979099, 74.79460685941484, 77.6160146, 0.09772783661369303, 74.7963208, 76.5066292, 77.63040639553275, 77.66676103753434, -71.6118577, 77.7640586, -109.4595323, 77.6404672, 121.070371, 77.6130165, 77.595715, 79.5967057, -76.964526, -76.373824, -0.1042366, 77.6246697, -77.5275749, 73.8499601124847, -78.974578, -0.7354457, -88.066644, 73.1597756, -0.073532, 77.58516877601824, 77.6323597, 77.6381958, 77.6269144, 77.6568367, 78.7011301, 77.5703253, 77.5753419, 77.6377094, 77.6203272, 103.84320655721689, 77.6037312, 77.5548006, 81.6048241, -79.0345627, 77.7150877, 77.7470942, 77.695935, 77.6255562, 77.6266252, 77.7519261, 77.7831871, 77.554883, 77.6518539, 77.7660547, 78.5541977, 77.6305067, 77.6

In [20]:
locations['Latitude'] = lat
locations['Longitude'] = lon
locations.head()

,Name,Latitude,Longitude
0,Banashankari,15.887678,75.704678
1,Basavanagudi,12.941726,77.575502
2,Mysore Road,12.387214,76.666963
3,Jayanagar,27.643927,83.052805
4,Kumaraswamy Layout,12.908149,77.555318


In [22]:
# converting dataframe in csv
locations.to_csv('Dataset/location.csv', index=False)

In [25]:
Rest_Location = df['location'].value_counts().reset_index()
Rest_Location.head()

,index,location
0,BTM,5124
1,HSR,2523
2,Koramangala 5th Block,2504
3,JP Nagar,2235
4,Whitefield,2144


In [26]:
# Name column is basically location and count column shows us amount of Restaurants in that location
Rest_Location.columns=['Name', 'Count']
Rest_Location.head()

,Name,Count
0,BTM,5124
1,HSR,2523
2,Koramangala 5th Block,2504
3,JP Nagar,2235
4,Whitefield,2144


In [27]:
# mearning locations and Res_Locations Dataframe and dropping na values
Restaurants_Locations = Rest_Location.merge(locations, on='Name', how='left').dropna()

In [28]:
Restaurants_Locations.head()

,Name,Count,Latitude,Longitude
0,BTM,5124,45.954851,-112.496595
1,HSR,2523,18.147500,41.538889
2,Koramangala 5th Block,2504,13.005409,74.794607
3,JP Nagar,2235,12.265594,76.646540
4,Whitefield,2144,44.373058,-71.611858


## Spatial Analysis on Restaurants_Locations DF

In [31]:
import folium

In [42]:
def generateBaseMap(defaul_location=[12.9716, 77.5946], default_zoom_start=12):
    baseMap = folium.Map(location=defaul_location, zoom_start=default_zoom_start)
    return baseMap

In [43]:
baseMap = generateBaseMap()
baseMap

#### Creating heatmap of all the restaurants in Restaurants_Locations DF

In [37]:
from folium.plugins import HeatMap

In [44]:
HeatMap(Restaurants_Locations[['Latitude', 'Longitude', 'Count']], zoom=20).add_to(baseMap)

In [45]:
baseMap

#### Marker Cluster

In [46]:
from folium.plugins import FastMarkerCluster 

In [47]:
FastMarkerCluster(Restaurants_Locations[['Latitude', 'Longitude', 'Count']], zoom=20).add_to(baseMap)

In [48]:
baseMap

In [49]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [51]:
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', nan, '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [52]:
df.dropna(axis=0, subset=['rate'], inplace=True)

In [53]:
# Removing "/5" from data
def split(x):
    return x.split('/')[0]

In [54]:
df['rating'] = df['rate'].apply(split)

In [55]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.1
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.8
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.7
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.8


In [56]:
df['rating'].unique()

array(['4.1', '3.8', '3.7', '3.6', '4.6', '4.0', '4.2', '3.9', '3.1',
       '3.0', '3.2', '3.3', '2.8', '4.4', '4.3', 'NEW', '2.9', '3.5',
       '2.6', '3.8 ', '3.4', '4.5', '2.5', '2.7', '4.7', '2.4', '2.2',
       '2.3', '3.4 ', '-', '3.6 ', '4.8', '3.9 ', '4.2 ', '4.0 ', '4.1 ',
       '3.7 ', '3.1 ', '2.9 ', '3.3 ', '2.8 ', '3.5 ', '2.7 ', '2.5 ',
       '3.2 ', '2.6 ', '4.5 ', '4.3 ', '4.4 ', '4.9', '2.1', '2.0', '1.8',
       '4.6 ', '4.9 ', '3.0 ', '4.8 ', '2.3 ', '4.7 ', '2.4 ', '2.1 ',
       '2.2 ', '2.0 ', '1.8 '], dtype=object)

In [60]:
df.replace('NEW', 1, inplace=True)

In [61]:
df.replace('-', 1, inplace=True)

In [62]:
df['rating'].unique()

array(['4.1', '3.8', '3.7', '3.6', '4.6', '4.0', '4.2', '3.9', '3.1',
       '3.0', '3.2', '3.3', '2.8', '4.4', '4.3', 1, '2.9', '3.5', '2.6',
       '3.8 ', '3.4', '4.5', '2.5', '2.7', '4.7', '2.4', '2.2', '2.3',
       '3.4 ', '3.6 ', '4.8', '3.9 ', '4.2 ', '4.0 ', '4.1 ', '3.7 ',
       '3.1 ', '2.9 ', '3.3 ', '2.8 ', '3.5 ', '2.7 ', '2.5 ', '3.2 ',
       '2.6 ', '4.5 ', '4.3 ', '4.4 ', '4.9', '2.1', '2.0', '1.8', '4.6 ',
       '4.9 ', '3.0 ', '4.8 ', '2.3 ', '4.7 ', '2.4 ', '2.1 ', '2.2 ',
       '2.0 ', '1.8 '], dtype=object)

In [64]:
df.dtypes

url                            object
address                        object
name                           object
online_order                   object
book_table                     object
rate                           object
votes                           int64
phone                          object
location                       object
rest_type                      object
dish_liked                     object
cuisines                       object
approx_cost(for two people)    object
reviews_list                   object
menu_item                      object
listed_in(type)                object
listed_in(city)                object
rating                         object
dtype: object

In [65]:
# converting rating column in numerical format
df['rating'] = pd.to_numeric(df['rating'])

In [68]:
df.groupby('location')['rating'].mean().sort_values(ascending=False)

location
Lavelle Road             4.066934
St. Marks Road           4.017201
Koramangala 3rd Block    3.989119
Church Street            3.970364
Sankey Road              3.965385
                           ...   
Electronic City          3.170539
Hebbal                   3.080000
Bommanahalli             3.009554
North Bangalore          2.671429
West Bangalore           2.420000
Name: rating, Length: 92, dtype: float64

In [70]:
avg_rating = df.groupby('location')['rating'].mean().sort_values(ascending=False).values

In [72]:
avg_rating

array([4.06693387, 4.01720117, 3.98911917, 3.97036364, 3.96538462,
       3.92755145, 3.90105263, 3.85      , 3.84950658, 3.84097222,
       3.83965517, 3.80740741, 3.79642857, 3.77447199, 3.77045455,
       3.74160584, 3.72271293, 3.72222222, 3.71486486, 3.70492958,
       3.6981405 , 3.69306931, 3.69255319, 3.6589057 , 3.64083333,
       3.64      , 3.63870968, 3.625369  , 3.625     , 3.61185468,
       3.60873016, 3.6       , 3.5991121 , 3.5959893 , 3.57871972,
       3.54552381, 3.53529135, 3.5312296 , 3.53073248, 3.5212766 ,
       3.51111111, 3.51111111, 3.50267857, 3.50068966, 3.49891697,
       3.48695652, 3.48461538, 3.48431585, 3.47894737, 3.47577002,
       3.47179487, 3.47175573, 3.46165803, 3.4497342 , 3.44906832,
       3.44175824, 3.44045243, 3.4375    , 3.43666093, 3.43216783,
       3.401002  , 3.4       , 3.4       , 3.39166667, 3.38738739,
       3.38181818, 3.38103627, 3.38      , 3.37419355, 3.37380897,
       3.37105751, 3.36190476, 3.35338983, 3.34788732, 3.33889

In [73]:
location = df.groupby('location')['rating'].mean().sort_values(ascending=False).index

In [74]:
location

Index(['Lavelle Road', 'St. Marks Road', 'Koramangala 3rd Block',
       'Church Street', 'Sankey Road', 'Koramangala 5th Block',
       'Cunningham Road', 'Rajarajeshwari Nagar', 'Residency Road',
       'Koramangala 4th Block', 'Sadashiv Nagar', 'Langford Town',
       'Infantry Road', 'Koramangala 7th Block', 'MG Road', 'Race Course Road',
       'Richmond Road', 'Kengeri', 'Hosur Road', 'Seshadripuram',
       'Indiranagar', 'Koramangala 6th Block', 'Malleshwaram', 'Jayanagar',
       'Koramangala 8th Block', 'Yelahanka', 'Koramangala', 'Brigade Road',
       'Central Bangalore', 'New BEL Road', 'Vasanth Nagar',
       'Koramangala 2nd Block', 'Ulsoor', 'Kalyan Nagar', 'Frazer Town',
       'Kammanahalli', 'HSR', 'Sarjapur Road', 'Basavanagudi',
       'Sahakara Nagar', 'Mysore Road', 'Uttarahalli', 'Yeshwantpur',
       'Basaveshwara Nagar', 'Shivajinagar', 'Jalahalli', 'Kaggadasapura',
       'JP Nagar', 'Kanakapura Road', 'Rajajinagar', 'City Market',
       'Jeevan Bhima Nagar'

In [75]:
rating = pd.DataFrame()

In [76]:
lat = []
lon = []
for loc in location:
    
    # converting string location into geo format
    loc = geolocator.geocode(loc)
    
    if loc is None:
        lat.append(np.nan)
        lon.append(np.nan)
    
    else:
        lat.append(loc.latitude)
        lon.append(loc.longitude)

In [77]:
rating['location'] = location
rating['Latitude'] = lat
rating['Longitude'] = lon
rating['Average Rating'] = avg_rating

In [78]:
rating.head()

,location,Latitude,Longitude,Average Rating
0,Lavelle Road,40.765284,-76.373824,4.066934
1,St. Marks Road,51.522765,-0.735446,4.017201
2,Koramangala 3rd Block,12.927187,77.626625,3.989119
3,Church Street,51.373656,-0.104237,3.970364
4,Sankey Road,38.780108,-121.505644,3.965385


In [79]:
rating.isna().sum()

location          0
Latitude          1
Longitude         1
Average Rating    0
dtype: int64

In [80]:
rating.dropna(inplace=True)

In [81]:
rating.isna().sum()

location          0
Latitude          0
Longitude         0
Average Rating    0
dtype: int64

In [82]:
HeatMap(rating[['Latitude', 'Longitude', 'Average Rating']]).add_to(baseMap)

In [83]:
baseMap

In [84]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.1
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.8
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.7
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.8


In [86]:
filter = df['cuisines']=="North Indian"
df2 = df[filter]
df2.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,No,3.8/5,286,+91 9980040002\r\n+91 9980063005,Basavanagudi,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[],Buffet,Banashankari,3.8
50,https://www.zomato.com/bangalore/petoo-banasha...,"276, Ground Floor, 100 Feet Outer Ring Road, B...",Petoo,No,No,3.7/5,21,+91 8026893211,Banashankari,Quick Bites,NaN,North Indian,450,"[('Rated 2.0', 'RATED\n This is a neatly made...",[],Delivery,Banashankari,3.7
72,https://www.zomato.com/bangalore/spicy-tandoor...,"Opposite ICICi Bank, Hanuman Nagar, Banashanka...",Spicy Tandoor,No,No,1,0,+91 8050884222,Banashankari,Quick Bites,NaN,North Indian,150,"[('Rated 4.0', 'RATED\n cost for chicken roll...",[],Delivery,Banashankari,1.0
87,https://www.zomato.com/bangalore/krishna-sagar...,"38, 22nd Main, 22nd Cross, Opposite BDA, 2nd S...",Krishna Sagar,No,No,3.5/5,31,+91 8892752997\r\n+91 7204780429,Banashankari,Quick Bites,NaN,North Indian,200,"[('Rated 1.0', 'RATED\n Worst experience with...",[],Delivery,Banashankari,3.5
94,https://www.zomato.com/bangalore/nandhini-delu...,"304, Opposite Apollo Public School, 100 Feet R...",Nandhini Deluxe,No,No,2.6/5,283,080 26890011\r\n080 26890033,Banashankari,Casual Dining,"Biryani, Chicken Guntur, Thali, Buttermilk, Ma...",North Indian,600,"[('Rated 3.0', 'RATED\n Ididnt like much.\n\n...",[],Delivery,Banashankari,2.6


In [89]:
North_India = df2.groupby('location')['url'].count().reset_index()

In [90]:
North_India.head()

,location,url
0,BTM,274
1,Banashankari,35
2,Banaswadi,9
3,Bannerghatta Road,60
4,Basavanagudi,17


In [92]:
North_India.columns=['Name', 'Count']

In [93]:
North_India.head()

,Name,Count
0,BTM,274
1,Banashankari,35
2,Banaswadi,9
3,Bannerghatta Road,60
4,Basavanagudi,17


In [95]:
North_India = North_India.merge(locations, on='Name', how='left').dropna()

In [98]:
NI = North_India[['Latitude', 'Longitude', 'Count']].head()

In [99]:
HeatMap(NI, zoom=20, radius=50).add_to(baseMap)

In [100]:
baseMap

### Automate Spatial Analysis

In [101]:
def heatMap_zone(zone):
    df2 = df[df['cuisines']==zone]
    zoneDF = df2.groupby('location')['url'].count().reset_index()
    zoneDF.columns=['Name', 'Count']
    zoneDF = zoneDF.merge(locations, on='Name', how='left').dropna()
    z = zoneDF[['Latitude', 'Longitude', 'Count']].head()
    HeatMap(z, zoom=20, radius=50).add_to(baseMap)
    return baseMap

In [102]:
df['cuisines'].unique()

array(['North Indian, Mughlai, Chinese', 'Chinese, North Indian, Thai',
       'Cafe, Mexican, Italian', ..., 'Tibetan, Nepalese',
       'North Indian, Street Food, Biryani',
       'North Indian, Chinese, Arabian, Momos'], dtype=object)

In [103]:
# executing function for Nepalese
heatMap_zone('Nepalese')